In [299]:
import selenium as sel
from selenium.webdriver.common.by import By
import pandas as pd
import time
import datetime
import base as bs
import ElTiempoFunciones as etf

import os as os
import csv
import json

In [300]:
f = open('config.json')
empresas = json.load(f)

if not os.path.exists('../data/raw/eltiempo.csv'):
    os.makedirs('../data/raw/eltiempo.csv')
    
#for i in empresas['LaRepublica']:
    

In [301]:
#Empresa con la cual vamos a extraer los articulos
# TODO Tener una lista que itere por todos lo clientes
empresa = input("Digite la empresa a extraer: ")

In [302]:
#cerar driver... MODIFICAR DEPENDIENDO DEL NAVEGADOR
driver = sel.webdriver.Edge()
driver.get(f'https://www.eltiempo.com/buscar?q={empresa}')
# time.sleep(2)
driver.implicitly_wait(10) #Nueva metodología de wait

In [303]:
# El número total de ventanas que tiene la búsqueda

total_num_ventanas = driver.find_element(By.XPATH,'//*[@id="main-container"]/div[16]/div[2]/div[2]/div[2]/div/div[11]/div/ul/li[10]/a').get_attribute('href')
print(total_num_ventanas)

https://www.eltiempo.com/buscar/2767?q=ecopetrol


In [304]:
# Recolecta datos hasta la ventana 20 del Tiempo
total_num_ventanas = f'https://www.eltiempo.com/buscar/2?q=ecopetrol'

In [305]:
titulares=[]
get_url = driver.current_url
pag_actual = 1

while(get_url != total_num_ventanas):
        buscar = driver.find_element(By.XPATH,'//*[@id="main-container"]/div[16]/div[2]')
        articulos = buscar.find_elements(By.CLASS_NAME,"listing")
        
        print(articulos)

        for articulos in articulos:
                aux = articulos.find_element(By.XPATH, './/h3[contains(@class, "title-container")]')
                url = aux.find_element(By.XPATH,'.//a').get_attribute('href')
                # print(url)

                if not(bs.existedb(url, "eltiempo")):

                        titulo = articulos.find_element(By.CLASS_NAME,"title-container").text
                        # print(titulo)
                        resumen = articulos.find_element(By.CLASS_NAME,"epigraph-container").text
                        # print(resumen)
                        fechaPub = articulos.find_element(By.CLASS_NAME,"published-at").text
                        # print(fechaPub)
                        tema = articulos.find_element(By.CLASS_NAME,"category").text
                        # print(tema)
                        

                        titulares.append({'Fecha Extraccion':datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
                                        'Titulo':titulo,
                                        'Fecha Publicacion':fechaPub,
                                        'Tema':tema,
                                        'URL':url,
                                        'Resumen':resumen,
                                        'Empresa':empresa})
        pag_actual=pag_actual+1
        lik = driver.get(f'https://www.eltiempo.com/buscar/{pag_actual}?q=ecopetrol')
        driver.implicitly_wait(10) #Nueva metodología de wait
        get_url = driver.current_url
# driver.quit()
# print(titulares)

[<selenium.webdriver.remote.webelement.WebElement (session="c7de229052deea54873344adbb9d622d", element="3fe7e253-e723-467c-bb2a-003fc21b7ccd")>, <selenium.webdriver.remote.webelement.WebElement (session="c7de229052deea54873344adbb9d622d", element="62654b0b-11ec-4858-8d60-653dd6a4d251")>, <selenium.webdriver.remote.webelement.WebElement (session="c7de229052deea54873344adbb9d622d", element="75518527-94b2-48c1-a177-738e20f3a1d8")>, <selenium.webdriver.remote.webelement.WebElement (session="c7de229052deea54873344adbb9d622d", element="3edf812a-1cab-4c9c-8b37-09d940ecb58f")>, <selenium.webdriver.remote.webelement.WebElement (session="c7de229052deea54873344adbb9d622d", element="3e3bb118-de0c-4990-93b4-7e83bafe9db0")>, <selenium.webdriver.remote.webelement.WebElement (session="c7de229052deea54873344adbb9d622d", element="99392abd-fafd-49ab-a2ad-6a77a75e2b09")>, <selenium.webdriver.remote.webelement.WebElement (session="c7de229052deea54873344adbb9d622d", element="a9a5ebbd-cb55-44ef-9e19-ea0c8a4e

In [306]:
buscar = driver.find_element(By.XPATH,'//*[@id="main-container"]/div[16]/div[2]')
articulos = buscar.find_elements(By.CLASS_NAME,"listing")

for articulos in articulos:
        if not(bs.existedb(url, "eltiempo")):
                titulo = articulos.find_element(By.CLASS_NAME,"title-container").text
                # print(titulo)
                resumen = articulos.find_element(By.CLASS_NAME,"epigraph-container").text
                # print(resumen)
                fechaPub = articulos.find_element(By.CLASS_NAME,"published-at").text
                # print(fechaPub)
                tema = articulos.find_element(By.CLASS_NAME,"category").text
                # print(tema)
                aux = articulos.find_element(By.XPATH, './/h3[contains(@class, "title-container")]')
                url = aux.find_element(By.XPATH,'.//a').get_attribute('href')
                # print(url)

                titulares.append({'Fecha Extraccion':datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
                                'Titulo':titulo,
                                'Fecha Publicacion':fechaPub,
                                'Tema':tema,
                                'URL':url,
                                'Resumen':resumen,
                                'Empresa':empresa})

In [307]:
columnas = ['Fecha Extraccion','Titulo', 'Fecha Publicacion','Resumen','URL','Empresa','Tema']

# Borra los duplicados
result = [] 
[result.append(x) for x in titulares if x not in result] 

# Para AGREGAR titulares a un archivo.
with open('../data/raw/eltiempo.csv', 'a', newline='', errors='ignore') as csv_file:
    dict_object = csv.DictWriter(csv_file, fieldnames=columnas) 
  
    dict_object.writerows(result)

#Para crear un NUEVO archivo con los titulares
# pd.json_normalize(titulares).to_csv(f'../data/raw/eltiempo.csv',index=False, encoding='latin-1', errors='ignore', columns=columnas)

In [308]:
# making data frame from csv file
# data = pd.read_csv(r'../data/raw/eltiempo.csv')

# # dropping ALL duplicate values
# data.drop_duplicates(subset="First Name",
#                      keep=False, inplace=True)
  
# # displaying data
# data